In [2]:
# importing modules
import pytesseract
from pytesseract import Output
import joblib
from PIL import Image
import cv2
import csv
import import_ipynb
from pytesseract import Output
from plagiarism_checker import plagiarism_check

importing Jupyter notebook from plagiarism_checker.ipynb


In [2]:
#Function to extract text from image
def image_to_text(x): 
    
    #reading image using opencv
    image = cv2.imread(x)
    
    #converting image into gray scale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # converting it to binary image by Thresholding
    # this step is require if you have colored image because if you skip this part
    # then tesseract won't able to detect text correctly and this will give incorrect result
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    
    #configuring parameters for tesseract
    custom_config = r'--oem 3 --psm 6'
    details = pytesseract.image_to_data(threshold_img,output_type=Output.DICT,config=custom_config, lang='eng')
    total_boxes = len(details['text'])
    for sequence_number in range(total_boxes):
        if int(details['conf'][sequence_number]) >30:
            (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number],  details['height'][sequence_number])
            threshold_img = cv2.rectangle(threshold_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    parse_text = []
    word_list = []
    last_word = ''
    for word in details['text']:
        if word!='':
            word_list.append(word)
            last_word = word
        if (last_word!='' and word == '') or (word==details['text'][-1]):
            parse_text.append(word_list)
            word_list = []
    return parse_text
    

In [3]:
#function to convert result into file
def textConversion(file, file2,n):
    f1 = image_to_text(file)
    with open('result_text1.txt',  'w', newline="",encoding='utf-8') as file1:
        csv.writer(file1, delimiter=" ").writerows(f1)
    f2 = image_to_text(file2)
    with open('result_text2.txt',  'w', newline="",encoding='utf-8') as file3:
        csv.writer(file3, delimiter=" ").writerows(f2)
    result = plagiarism_check(n)
    return result